In [ ]:
!git clone https://github.com/Anushka-De/stat359.git
%cd stat359/student/Final_Project
!ls

Cloning into 'stat359'...
remote: Enumerating objects: 371, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 371 (delta 137), reused 93 (delta 88), pack-reused 202 (from 1)
Receiving objects: 100% (371/371), 6.01 MiB | 17.23 MiB/s, done.
Resolving deltas: 100% (204/204), done.
/content/stat359/student/Final_Project
arithmetic_llm	Proposal.docx


In [ ]:
!pip -q install numpy pandas gensim torch scikit-learn matplotlib ipywidgets nltk tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.6 MB/s eta 0:00:00


In [ ]:
!mkdir -p arithmetic_llm/data
!mkdir -p arithmetic_llm/models
!mkdir -p arithmetic_llm/evaluation_results
!mkdir -p arithmetic_llm/analysis
!mkdir -p arithmetic_llm/experiments


In [ ]:
# 1. Generate training corpus (100,000 samples recommended)
# Generate foundational training corpus with 100K samples (plain text)
# This large corpus provides the base model with extensive arithmetic patterns
!python -m arithmetic_llm.generate_foundational_plaintext \
  --num-samples 100000 \
  --max-depth 4 \
  --num-range 1 20 \
  --invalid-rate 0.05 \
  --output-txt arithmetic_llm/data/foundational_corpus.txt

# Generate mixed instruction corpus (valid + invalid)
# This creates a balanced dataset without writing intermediate files
!python -m arithmetic_llm.generate_instruction_corpus_mixed \
  --num-samples 20000 \
  --max-depth 4 \
  --num-range 1 20 \
  --invalid-rate 0 \
  --output-mixed arithmetic_llm/data/instruction_corpus.txt

# Generate separate test corpus for evaluation (10K samples, minimal errors)
# This provides a clean test set with only 1% invalid expressions
!python -m arithmetic_llm.generate_corpus \
  --instruction-only \
  --num-samples 1000 \
  --max-depth 4 \
  --output-instruction arithmetic_llm/data/instruction_corpus_test.txt \
  --num-range 1 20 \
  --invalid-rate 0

#check line counts
!python -c "import sys; [print(f'{sum(1 for _ in open(f))} {f}') for f in ['arithmetic_llm/data/foundational_corpus.txt', 'arithmetic_llm/data/instruction_corpus.txt', 'arithmetic_llm/data/instruction_corpus_test.txt']]"



Generating instruction corpus with 1000 samples...
Instruction corpus saved to: arithmetic_llm/data/instruction_corpus_test.txt
Corpus generation complete!
200000 arithmetic_llm/data/foundational_corpus.txt
40000 arithmetic_llm/data/instruction_corpus.txt
1000 arithmetic_llm/data/instruction_corpus_test.txt


In [ ]:
#2. Tokenizer Traning
!python -m arithmetic_llm.train_tokenizer \
  --corpus-path arithmetic_llm/data/foundational_corpus.txt \
  --vocab-size 1000 \
  --output-dir arithmetic_llm/data/tokenizer

# show tokenizer table
!python -m arithmetic_llm.print_token_table \
  --tokenizer_path arithmetic_llm/data/tokenizer/tokenizer.pkl \
  > tokens.csv

# Analyze your instruction corpus
!python -m arithmetic_llm.check_sequence_lengths \
  --corpus-path arithmetic_llm/data/instruction_corpus.txt \
  --tokenizer-path arithmetic_llm/data/tokenizer


Training BPE tokenizer with vocabulary size 1000...
Corpus: arithmetic_llm/data/foundational_corpus.txt
Building corpus: 200000it [00:07, 26710.35it/s]
BPE merges:  19% 186/1000 [00:00<00:00, 4371.34it/s]
Saving tokenizer to: arithmetic_llm/data/tokenizer

Tokenizer Statistics:
  Vocabulary size: 274
  BPE merge operations: 186
  Special tokens: <pad>, <unk>, <bos>, <eos>, <think>, </think>

Test encoding:
  Input: 5 + 10 - 3
  Encoded (with BOS/EOS): [174, 120, 8, 28, 11, 96, 175]
  Decoded: <bos> 5 + 10 - 3 <eos>
  Encoded (without BOS/EOS): [120, 8, 28, 11, 96]

Test encoding:
  Input: 12 - (4 + 2)
  Encoded (with BOS/EOS): [174, 49, 11, 4, 108, 8, 84, 6, 175]
  Decoded: <bos> 12 - ( 4 + 2 ) <eos>
  Encoded (without BOS/EOS): [49, 11, 4, 108, 8, 84, 6]

Test encoding:
  Input: ((7+3)-(3+5))
  Encoded (with BOS/EOS): [174, 4, 4, 144, 8, 96, 6, 11, 4, 96, 8, 120, 6, 6, 175]
  Decoded: <bos> ( ( 7 + 3 ) - ( 3 + 5 ) ) <eos>
  Encoded (without BOS/EOS): [4, 4, 144, 8, 96, 6, 11, 4, 96, 8

In [ ]:
!ls arithmetic_llm/data


foundational_corpus.txt      instruction_corpus.txt
instruction_corpus_test.txt  tokenizer


In [ ]:
# 3. Train foundational model
!python -m arithmetic_llm.run_foundational_training \
  --corpus-path arithmetic_llm/data/foundational_corpus.txt \
  --output-dir arithmetic_llm/models \
  --tokenizer-path arithmetic_llm/data/tokenizer \
  --num-epochs 10 \
  --max-seq-length 512 \
  --batch-size 16 \
  --device auto



FOUNDATIONAL MODEL TRAINING

Corpus: arithmetic_llm/data/foundational_corpus.txt
Tokenizer: arithmetic_llm/data/tokenizer
Output directory: arithmetic_llm/models

Training Configuration:
  Learning rate: 0.0001
  Batch size: 16
  Epochs: 10
  Warmup steps: 1000
  Gradient clip: 1.0
  Save every: 1000 steps
  Device: cpu

Model Configuration:
  d_model: 256
  nhead: 8
  num_layers: 6
  dim_feedforward: 1024
  dropout: 0.1
  max_seq_length: 512

Training output directory: arithmetic_llm/models/foundational_20260219_234605_495185
Configuration: {'learning_rate': 0.0001, 'batch_size': 16, 'num_epochs': 10, 'warmup_steps': 1000, 'gradient_clip': 1.0, 'save_every': 1000, 'eval_every': 500, 'device': 'cpu', 'lora_config': None}
Loading tokenizer...
Tokenizer vocabulary size: 271
Initializing model configuration...
Creating dataloaders...
Training batches: 11250
Validation batches: 1250
Initializing model...
Model parameters: 4,939,520

Starting training...

Epoch 1/10
Epoch 1:   5% 539/11250

In [ ]:
#3.1 Evaluate the foundational model, performance would be bad
!python -m arithmetic_llm.run_evaluation \
  --model-path arithmetic_llm/models/foundational_20260219_182844_242048/best_model.pt \
  --tokenizer-path arithmetic_llm/data/tokenizer \
  --max-gen-length 512 \
  --batch-size 1 \
  --num-samples 100 \
  --output-dir arithmetic_llm/evaluation_results/foundational_eval


MODEL EVALUATION

Model: arithmetic_llm/models/foundational_20260219_182844_242048/best_model.pt
Tokenizer: arithmetic_llm/data/tokenizer
Device: cuda

Evaluation Configuration:
  Test samples: 100
  Max depth: 5
  Number range: 1 to 20
  Batch size: 1
  Max generation length: 512
  Output directory: arithmetic_llm/evaluation_results/foundational_eval

Loading model and tokenizer...
Model loaded successfully!

Starting evaluation...
Generating 100 test expressions...
Generated 100 valid test expressions
Evaluating model with batch size 1...
Evaluated 1/100 samples
Evaluated 2/100 samples
Evaluated 3/100 samples
Evaluated 4/100 samples
Evaluated 5/100 samples
Evaluated 6/100 samples
Evaluated 7/100 samples
Evaluated 8/100 samples
Evaluated 9/100 samples
Evaluated 10/100 samples
Evaluated 11/100 samples
Evaluated 12/100 samples
Evaluated 13/100 samples
Evaluated 14/100 samples
Evaluated 15/100 samples
Evaluated 16/100 samples
Evaluated 17/100 samples
Evaluated 18/100 samples
Evaluated 1

In [ ]:
!ls arithmetic_llm/models

foundational_20260219_182844_242048


In [ ]:
# 4. Fine-tune instruction model
!python -m arithmetic_llm.run_instruction_training \
  --instruction-corpus-path arithmetic_llm/data/instruction_corpus.txt \
  --output-dir arithmetic_llm/models \
  --tokenizer-path arithmetic_llm/data/tokenizer \
  --foundational-checkpoint arithmetic_llm/models/foundational_20260219_182844_242048/best_model.pt \
  --num-epochs 10 \
  --batch-size 16 \
  --device auto


INSTRUCTION FINE-TUNING

Instruction corpus: arithmetic_llm/data/instruction_corpus.txt
Tokenizer: arithmetic_llm/data/tokenizer
Foundational checkpoint: arithmetic_llm/models/foundational_20260219_182844_242048/best_model.pt
Output directory: arithmetic_llm/models

Training Configuration:
  Learning rate: 5e-05
  Batch size: 16
  Epochs: 10
  Warmup steps: 500
  Gradient clip: 1.0
  Save every: 500 steps
  Device: cuda

Fine-tuning output directory: arithmetic_llm/models/instruction_20260219_220640_551798
Configuration: {'learning_rate': 5e-05, 'batch_size': 16, 'num_epochs': 10, 'warmup_steps': 500, 'gradient_clip': 1.0, 'save_every': 500, 'eval_every': 500, 'device': 'cuda', 'lora_config': None}
Loading tokenizer...
Tokenizer vocabulary size: 274
Initializing model architecture...
Creating dataloaders...
Training batches: 2250
Validation batches: 250
Loading foundational model from: arithmetic_llm/models/foundational_20260219_182844_242048/best_model.pt
Loaded checkpoint from epoch

In [ ]:
!ls arithmetic_llm/models

foundational_20260219_182844_242048  instruction_20260219_220640_551798


In [ ]:
# 4.1 Evaluate the model
!python -m arithmetic_llm.run_evaluation \
  --model-path arithmetic_llm/models/instruction_20260219_220640_551798/best_model.pt \
  --tokenizer-path arithmetic_llm/data/tokenizer \
  --max-gen-length 512 \
  --batch-size 1 \
  --num-samples 1000 \
  --output-dir arithmetic_llm/evaluation_results/instruction_id_eval


MODEL EVALUATION

Model: arithmetic_llm/models/instruction_20260219_220640_551798/best_model.pt
Tokenizer: arithmetic_llm/data/tokenizer
Device: cuda

Evaluation Configuration:
  Test samples: 1000
  Max depth: 5
  Number range: 1 to 20
  Batch size: 1
  Max generation length: 512
  Output directory: arithmetic_llm/evaluation_results/instruction_id_eval

Loading model and tokenizer...
Model loaded successfully!

Starting evaluation...
Generating 1000 test expressions...
Generated 1000 valid test expressions
Evaluating model with batch size 1...
Evaluated 1/1000 samples
Evaluated 2/1000 samples
Evaluated 3/1000 samples
Evaluated 4/1000 samples
Evaluated 5/1000 samples
Evaluated 6/1000 samples
Evaluated 7/1000 samples
Evaluated 8/1000 samples
Evaluated 9/1000 samples
Evaluated 10/1000 samples
Evaluated 11/1000 samples
Evaluated 12/1000 samples
Evaluated 13/1000 samples
Evaluated 14/1000 samples
Evaluated 15/1000 samples
Evaluated 16/1000 samples
Evaluated 17/1000 samples
Evaluated 18/10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p /content/drive/MyDrive/arithmetic_llm_backup

!cp -r arithmetic_llm/data/tokenizer \
      arithmetic_llm/models \
      arithmetic_llm/evaluation_results \
      /content/drive/MyDrive/arithmetic_llm_backup/

cp: cannot stat 'arithmetic_llm/data/tokenizer': No such file or directory
cp: cannot stat 'arithmetic_llm/models': No such file or directory
